In [ ]:
# default_exp models.ROCKET

# ROCKET

> ROCKET (RandOm Convolutional KErnel Transform) functions for univariate and multivariate time series.

In [ ]:
#export
try: 
    import sktime
    from sktime.transformations.panel.rocket import Rocket
except ImportError:
    print("You need to install sktime to be able to use RocketClassifier and RocketRegressor")

In [ ]:
#export
import sklearn
from sklearn.linear_model import RidgeClassifierCV, RidgeCV
from sklearn.metrics import make_scorer
from tsai.imports import *
from tsai.data.external import *
from tsai.models.layers import *
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
#export
class RocketClassifier(sklearn.pipeline.Pipeline):
    """Time series classification using ROCKET features and a linear classifier"""
    
    def __init__(self, num_kernels=10_000, normalize_input=True, random_state=None, 
                 alphas=np.logspace(-3, 3, 7), normalize_features=True, memory=None, verbose=False, scoring=None, class_weight=None, **kwargs):
        """
        RocketClassifier is recommended for up to 10k time series. 
        For a larger dataset, you can use ROCKET (in Pytorch).
        scoring = None --> defaults to accuracy.
        
        Rocket args:            
            num_kernels     : int, number of random convolutional kernels (default 10,000)
            normalize_input : boolean, whether or not to normalise the input time series per instance (default True)
            random_state    : Optional random seed (default None)

        """

        self.steps = [('rocket', Rocket(num_kernels=num_kernels, normalise=normalize_input, random_state=random_state)),
                      ('ridgeclassifiercv', RidgeClassifierCV(alphas=alphas, normalize=normalize_features, scoring=scoring, 
                                                              class_weight=class_weight, **kwargs))]
        store_attr()
        self._validate_steps()

    def __repr__(self):  
        return f'Pipeline(steps={self.steps.copy()})'

    def save(self, fname='Rocket', path='./models'):
        path = Path(path)
        filename = path/fname
        with open(f'{filename}.pkl', 'wb') as output:
            pickle.dump(self, output, pickle.HIGHEST_PROTOCOL)

In [ ]:
#export
def load_rocket(fname='Rocket', path='./models'):
    path = Path(path)
    filename = path/fname
    with open(f'{filename}.pkl', 'rb') as input:
        output = pickle.load(input)
    return output

In [ ]:
#export
class RocketRegressor(sklearn.pipeline.Pipeline):
    """Time series regression using ROCKET features and a linear regressor"""
    
    def __init__(self, num_kernels=10_000, normalize_input=True, random_state=None, 
                 alphas=np.logspace(-3, 3, 7), normalize_features=True, memory=None, verbose=False, scoring=None, **kwargs):
        """
        RocketRegressor is recommended for up to 10k time series. 
        For a larger dataset, you can use ROCKET (in Pytorch).
        scoring = None --> defaults to r2.
        
        Args:            
            num_kernels     : int, number of random convolutional kernels (default 10,000)
            normalize_input : boolean, whether or not to normalise the input time series per instance (default True)
            random_state    : Optional random seed (default None)
        """

        self.steps = [('rocket', Rocket(num_kernels=num_kernels, normalise=normalize_input, random_state=random_state)),
                  ('ridgecv', RidgeCV(alphas=alphas, normalize=normalize_features, scoring=scoring, **kwargs))]
        store_attr()
        self._validate_steps()


    def __repr__(self):  
        return f'Pipeline(steps={self.steps.copy()})'

    def save(self, fname='Rocket', path='./models'):
        path = Path(path)
        filename = path/fname
        with open(f'{filename}.pkl', 'wb') as output:
            pickle.dump(self, output, pickle.HIGHEST_PROTOCOL)

In [ ]:
#extras
# Univariate classification with sklearn-type API
dsid = 'OliveOil'
fname = 'RocketClassifier'
X_train, y_train, X_test, y_test = get_UCR_data(dsid, Xdtype='float64')
cls = RocketClassifier()
cls.fit(X_train, y_train)
cls.save(fname)
del cls
cls = load_rocket(fname)
print(cls.score(X_test, y_test))

0.9


In [ ]:
#extras
# Multivariate classification with sklearn-type API
dsid = 'NATOPS'
fname = 'RocketClassifier'
X_train, y_train, X_test, y_test = get_UCR_data(dsid, Xdtype='float64')
cls = RocketClassifier()
cls.fit(X_train, y_train)
cls.save(fname)
del cls
cls = load_rocket(fname)
print(cls.score(X_test, y_test))

In [ ]:
#extras
# Univariate regression with sklearn-type API
from sklearn.metrics import mean_squared_error
dsid = 'Covid3Month'
fname = 'RocketRegressor'
X_train, y_train, X_test, y_test = get_Monash_regression_data(dsid, Xdtype='float64')
if X_train is not None: 
    rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
    reg = RocketRegressor(scoring=rmse_scorer)
    reg.fit(X_train, y_train)
    reg.save(fname)
    del reg
    reg = load_rocket(fname)
    y_pred = reg.predict(X_test)
    print(mean_squared_error(y_test, y_pred, squared=False))

In [ ]:
#extras
# Multivariate regression with sklearn-type API
from sklearn.metrics import mean_squared_error
dsid = 'AppliancesEnergy'
fname = 'RocketRegressor'
X_train, y_train, X_test, y_test = get_Monash_regression_data(dsid, Xdtype='float64')
if X_train is not None: 
    rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
    reg = RocketRegressor(scoring=rmse_scorer)
    reg.fit(X_train, y_train)
    reg.save(fname)
    del reg
    reg = load_rocket(fname)
    y_pred = reg.predict(X_test)
    print(mean_squared_error(y_test, y_pred, squared=False))

In [ ]:
#hide
from tsai.imports import *
from tsai.export import *
nb_name = get_nb_name()
# nb_name = "111_models.ROCKET.ipynb"
create_scripts(nb_name);